# QuantumExecutor Demo Notebook

This notebook demonstrates how to use the `QuantumExecutor` class, which provides a unified interface for:

- Splitting quantum jobs,
- Dispatching them to different quantum backends,
- Executing them either sequentially or in parallel,
- And finally, aggregating the results.

QuantumExecutor offers two ways to determine how to split and dispatch the jobs:

1. **Dispatch Mode** – Enables to define directly a custom `Dispatch` object to manage multiple quantum circuits, enabling fine-grained control over their independent and parallel execution on different quantum backends.
2. **Experiment Mode** – Determines dynamically how to dispatch the jobs through a split policy.

Each modality supports four execution strategies:

- **Synchronous Execution (non-blocking)** – Tasks still run one after the other in a demon thread without blocking the main thread.
- **Synchronous Execution (blocking)** – Tasks run one after the other in the main thread.
- **Asynchronous Execution (non-blocking)** – Tasks are dispatched using multiprocessing and do not block the main thread.
- **Asynchronous Execution (blocking)** – Tasks run in parallel but the main thread waits until all are completed.

Note: The term "quantum backend" refers to a quantum computer or simulator provided by platforms like IBM Quantum, IonQ, etc.

> 💡 Before proceeding, make sure the required dependencies are installed: `qiskit`, `qbraid`, and other supporting libraries!

## 1. Setup and Imports

We begin by importing the required libraries and setting up the environment.

In [1]:
# Import the QuantumExecutor class from the quantum_executor module
# Import json for pretty printing
import json

# Import Qiskit components for circuit creation
from qiskit import QuantumCircuit

from quantum_executor import Dispatch
from quantum_executor import QuantumExecutor

## 2. Initialize the QuantumExecutor

We now instantiate the `QuantumExecutor`. In this example, we just use the local provider.

To know the available providers we can use the static method `default_providers()`

In [2]:
QuantumExecutor.default_providers()

['azure', 'braket', 'ionq', 'local_aer', 'qbraid', 'qiskit']

In [3]:
# Instantiate the QuantumExecutor.
qe = QuantumExecutor(providers=["local_aer"])
print("Initialized providers:", qe.providers)

Initialized providers: {'local_aer': <quantum_executor.local_aer.provider.LocalAERProvider object at 0x168ae7710>}


## 2. Creating a Two Quantum Circuits

We create two simple 2-qubit quantum circuits that prepares two Bell states.

In [4]:
# Create two simple 2-qubit quantum circuits that creates a Bell states
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()
qc.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

In [5]:
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.h(1)
qc1.cx(0, 1)
qc1.measure_all()
qc1.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        ├───┤┌─┴─┐ ░ └╥┘┌─┐
   q_1: ┤ H ├┤ X ├─░──╫─┤M├
        └───┘└───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

## 3. Running a Dispatch

To have complete control of the backends configuration and of the execution process it is possible to directly run a custom `Dispatch`. With a `Dispatch` it is possible to add different jobs to be run together on multiple quantum computers (and/or on the same backend multiple times). Moreover it is possible, whenever the quantum device allows it, to specify custom configuration of the backend.

In this demo we are going to run two circuits on two backend (the first will run both, the second just one of them), and we are going to specify a seed for the execution

In [6]:
dispatch = Dispatch()
dispatch.add_job(
    provider_name="local_aer",
    backend_name="fake_torino",
    circuits=qc,
    shots=2048,
    config={"seed": 42},
)
dispatch.add_job(
    provider_name="local_aer",
    backend_name="fake_oslo",
    circuits=qc1,
    shots=2048,
    config={"seed": 42},
)
dispatch.add_job(
    provider_name="local_aer",
    backend_name="fake_torino",
    circuits=qc1,
    shots=1024,
    config={"seed": 41},
)

dispatch

Dispatch({'local_aer': {'fake_torino': [Job(id=209a24a1-64f7-420c-abea-157b37afcd9e, circuit_type=QuantumCircuit, shots=2048, config={'seed': 42}), Job(id=6f558a93-dc5a-4903-87dc-e0c87ee354db, circuit_type=QuantumCircuit, shots=1024, config={'seed': 41})], 'fake_oslo': [Job(id=bdb1b5b9-f44f-40af-ab90-ce8fc784d9bb, circuit_type=QuantumCircuit, shots=2048, config={'seed': 42})]}})

However, equivalently, it is available an easier, low code way to define a dispatch through a dictionary:

In [7]:
dispatch = {
    "local_aer": {
        "fake_torino": [
            {
                "circuit": qc,
                "shots": 2048,
                "config": {"seed": 42},
            },
            {
                "circuit": qc1,
                "shots": 1024,
                "config": {"seed": 41},
            },
        ],
        "fake_oslo": [
            {
                "circuit": qc1,
                "shots": 2048,
                "config": {"seed": 42},
            },
        ],
    }
}

dispatch

{'local_aer': {'fake_torino': [{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x168ae7dd0>,
    'shots': 2048,
    'config': {'seed': 42}},
   {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1693cc410>,
    'shots': 1024,
    'config': {'seed': 41}}],
  'fake_oslo': [{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1693cc410>,
    'shots': 2048,
    'config': {'seed': 42}}]}}

We can run a `Dispatch` synchronously in the main thread specifying `multiprocess=False` and `wait=True`

In [8]:
res = qe.run_dispatch(
    dispatch=dispatch,
    multiprocess=False,
    wait=True,
)

res

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=b583bc09-dc91-44c0-9ffa-33061f60dc69, circuit_type=QuantumCircuit, shots=2048, config={}), status=Complete, data={'00': 1034, '01': 20, '10': 19, '11': 975}), JobResult(job=Job(id=009e8e0e-b37a-4166-b6ac-764e4b21eb22, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 262, '01': 249, '10': 255, '11': 258})], 'fake_oslo': [JobResult(job=Job(id=f452d797-7c89-4749-90bd-0c32de26bbb1, circuit_type=QuantumCircuit, shots=2048, config={}), status=Complete, data={'00': 514, '01': 537, '10': 471, '11': 526})]}})

Or synchronously but in a daemon thread with `multiprocess=False` and `wait=False`

In [9]:
res1 = qe.run_dispatch(
    dispatch=dispatch,
    multiprocess=False,
    wait=False,
)

res1

ResultCollector(complete_results=0, total_results=3, complete=False)

In this case, the `run_dispatch` function ends immediately and we can poll whenever we want the results, as soon as a result became available we can already using it even if other quantum computer are still not retuning the results.

In [10]:
import time

while not res1.complete:
    print("Waiting for results...")
    print(res1)
    print(json.dumps(res1.get_results(), indent=4))
    time.sleep(1)
print("Results are ready!")

Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCol

In [11]:
print(json.dumps(res1.get_results(), indent=4))

{
    "local_aer": {
        "fake_torino": [
            {
                "00": 1007,
                "01": 31,
                "10": 15,
                "11": 995
            },
            {
                "00": 256,
                "01": 254,
                "10": 234,
                "11": 280
            }
        ],
        "fake_oslo": [
            {
                "00": 496,
                "01": 502,
                "10": 493,
                "11": 557
            }
        ]
    }
}


We can run a `Dispatch` asynchronously with `multiprocess=True`, in this case it is possible to specify if we want a blocking call waiting for the results with `wait=True`

In [12]:
res2 = qe.run_dispatch(dispatch=dispatch, multiprocess=True, wait=True)

res2

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=c12e335b-5af2-4967-b5bd-0743e98118d6, circuit_type=QuantumCircuit, shots=2048, config={}), status=Complete, data={'00': 1024, '01': 33, '10': 17, '11': 974}), JobResult(job=Job(id=98c89b82-28b9-42ec-9f80-9cbdf88a42e6, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 261, '01': 253, '10': 253, '11': 257})], 'fake_oslo': [JobResult(job=Job(id=cc07301d-fb46-447c-8a54-91b0b0caacd7, circuit_type=QuantumCircuit, shots=2048, config={}), status=Complete, data={'00': 554, '01': 463, '10': 508, '11': 523})]}})

Or with a non-blocking call with `wait=True`. 

In [13]:
res3 = qe.run_dispatch(dispatch=dispatch, multiprocess=True, wait=False)

res3

ResultCollector(complete_results=0, total_results=3, complete=False)

In [14]:
while not res3.complete:
    print("Waiting for results...")
    print(res3)
    print(json.dumps(res3.get_results(), indent=4))
    time.sleep(1)
print("Results are ready!")

Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
ResultCollector(complete_results=1, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            {
                "00": 543,
                "01": 473,

In [15]:
print(json.dumps(res3.get_results(), indent=4))

{
    "local_aer": {
        "fake_torino": [
            {
                "00": 1022,
                "01": 22,
                "10": 23,
                "11": 981
            },
            {
                "00": 277,
                "01": 255,
                "10": 260,
                "11": 232
            }
        ],
        "fake_oslo": [
            {
                "00": 543,
                "01": 473,
                "10": 535,
                "11": 497
            }
        ]
    }
}


## 4. Aggregating the Results

Through the QuantumExecutor, it is also possible to dynamically and automatically analyse and aggregate the output data. This is done through a `merge` function which can be (optionally) specified to run automatically after all the result are aggregated.

### 4.1 Running a Default policy
First, we will show how to use one of the pre-defined policies, the how it is possible to define a custom `merge` policy.

Run the aggregation policy is easy and straightforward, it is just an additional parameter in the run function.

In [16]:
res4 = qe.run_dispatch(dispatch=dispatch, merge_policy="simple_aggregate", multiprocess=True, wait=False)

res4

MergedResultCollector(complete_results=0, total_results=3, complete=False)

The `simple-aggregate` policy just sums together all the output of the quantum backends, different policies can be specified or add to the QuantumExecutor. While the dispatch is running, it is still possible to access the partial results.

> Note that the merge policy is independent from the execution modes, i.e., it can be used both with and without multiprocessing

In [17]:
while not res4.complete:
    print("Waiting for results...")
    print(res4)
    print(json.dumps(res4.get_results(), indent=4))
    time.sleep(1)
print("Results are ready!")

Waiting for results...
MergedResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
MergedResultCollector(complete_results=0, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            null
        ]
    }
}
Waiting for results...
MergedResultCollector(complete_results=1, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torino": [
            null,
            null
        ],
        "fake_oslo": [
            {
                "00": 505,
                "01": 480,
                "10": 584,
                "11": 479
            }
        ]
    }
}
Waiting for results...
MergedResultCollector(complete_results=1, total_results=3, complete=False)
{
    "local_aer": {
        "fake_torin

In [18]:
print(json.dumps(res4.get_results(), indent=4))

{
    "local_aer": {
        "fake_torino": [
            {
                "00": 1001,
                "01": 29,
                "10": 15,
                "11": 1003
            },
            {
                "00": 255,
                "01": 247,
                "10": 266,
                "11": 256
            }
        ],
        "fake_oslo": [
            {
                "00": 505,
                "01": 480,
                "10": 584,
                "11": 479
            }
        ]
    }
}


In [19]:
res4

MergedResultCollector(merged_results={'00': 1761, '01': 756, '10': 865, '11': 1738}, initial_policy_data={}, final_policy_data={})

We can get the merged results with

In [20]:
res4.get_merged_results()

{'00': 1761, '01': 756, '10': 865, '11': 1738}

### 4.2 Running a Custom policy

Now we are going to develop a policy which first sums all the contributions from the different quantum backends and the select the state with the maximum frequency, a merge policy requires in input the dictionary of results some optional additional data (which we will not use in this demo).

In [21]:
from typing import Any

from quantum_executor import Dispatch
from quantum_executor import VirtualProvider
from quantum_executor.job_runner import ResultData


def merge(
    results: dict[str, dict[str, list[ResultData]]],
    policy_data: Any,  # noqa: ANN401
) -> tuple[dict[str, int], Any]:
    """
    Merge job results by summing bitstring counts, the return the state with the maximum count.

    Parameters
    ----------
    results : Dict[str, Dict[str, List[ResultData]]]
        Nested mapping (provider -> backend -> list of ResultData objects).
    policy_data : Any
        Additional data carried along; not used in this policy.

    Returns
    -------
    Tuple[Dict, Any]
        A tuple containing the merged counts dictionary and the unchanged blob.

    """
    merged_results: dict[str, int] = {}
    for _, provider_results in results.items():
        for _, job_results in provider_results.items():
            for result_data in job_results:
                if result_data is not None and isinstance(result_data, dict):
                    for bitstring, count in result_data.items():
                        if isinstance(count, int):
                            merged_results[bitstring] = merged_results.get(bitstring, 0) + count

    # Find the bitstring with the maximum count
    max_bitstring = max(merged_results, key=merged_results.get)
    max_count = merged_results[max_bitstring]
    merged_results = {max_bitstring: max_count}

    return merged_results, policy_data

We can now register this policy on the `QuantumExecutor`:

In [22]:
qe.add_policy(name="max_frequency", merge_policy=merge)

And then run the circuit. This time we run the experiment asynchronously.

In [23]:
res5 = qe.run_dispatch(dispatch=dispatch, merge_policy="max_frequency", multiprocess=True, wait=False)

res5

MergedResultCollector(complete_results=0, total_results=3, complete=False)

In [24]:
res5.wait_for_completion()

True

In [25]:
res5

MergedResultCollector(merged_results={'00': 1851}, initial_policy_data={}, final_policy_data={})

In [26]:
res5.get_merged_results()

{'00': 1851}

## 5. Running an Experiment through a Dynamic Dispatch Policy

Finally, for more sophisticated experimentation is instead possible to define dynamically the Dispatch to run through a split policy.

### 5.1 Using a Default Policy
Similarly to the merge policy, there are some default policies like, the uniform policy which splits equally the shots of a circuit among all the quantum backends specified, of course more sophisticated polciies can be defined according to the specific use case.

First, we have to specify the backends we want to employ.

In [27]:
backends = {"local_aer": ["fake_torino", "fake_oslo"]}

backends = {
    "local": ["fake_torino", "fake_oslo"]
}

Now, we can run our experiment.

In [28]:
res6 = qe.run_experiment(
    circuit=qc,
    shots=1024,
    backends=backends,
    split_policy="uniform",
    wait=True,
)

res6

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=84eed947-76ac-4f15-b2ee-2ee8b7959416, circuit_type=QuantumCircuit, shots=512, config={}), status=Complete, data={'00': 235, '01': 9, '10': 4, '11': 264})], 'fake_oslo': [JobResult(job=Job(id=1b33c0fd-58f8-4a8e-a9b1-227912dbac9c, circuit_type=QuantumCircuit, shots=512, config={}), status=Complete, data={'00': 239, '01': 3, '10': 6, '11': 264})]}})

In [29]:
print(json.dumps(res6.get_results(), indent=4))

{
    "local_aer": {
        "fake_torino": [
            {
                "00": 235,
                "01": 9,
                "10": 4,
                "11": 264
            }
        ],
        "fake_oslo": [
            {
                "00": 239,
                "01": 3,
                "10": 6,
                "11": 264
            }
        ]
    }
}


### 5.2 Running a Custom policy
In this demo are going to develop a `custom-multiplier` policy (which mimics the behaviour of the `multiplier` policy already available among the default policies). Such a policy is the complimentary of the `uniform` policy, that is, it runs the same number of shots on each specified backend. 

A split policy requires the circuit and the associated number of shots, as well as the backends to employ and optional additional policy data, finally the virtual provider (see the relative) demo, it is also passed.

In [30]:
from typing import Any

from quantum_executor import Dispatch
from quantum_executor.job_runner import ResultData


def split(
    circuit: Any,  # noqa: ANN401
    shots: int,
    backends: dict[str, list[str]],
    policy_data: Any | None = None,  # noqa: ANN401
    _virtual_provider: VirtualProvider | None = None,
) -> tuple[Dispatch, Any]:
    """
    Run the same circuit and shot count on every backend.

    Parameters
    ----------
    circuit : Any
        The quantum circuit to run.
    shots : int
        Number of shots for the circuit.
    backends : Dict[str, List[str]]
        A dictionary mapping provider names to lists of backend names.
    policy_data : Any, optional
        Additional data carried along; not used in this policy.
    _virtual_provider : Optional[VirtualProvider], optional
        An instance of VirtualProvider; not used in this policy.

    Returns
    -------
    Tuple[Dispatch, Any]
        A tuple containing the Dispatch object with registered jobs and the unchanged blob.

    """
    dispatch = Dispatch()
    for provider_name, backends_ls in backends.items():
        for backend_name in backends_ls:
            dispatch.add_job(
                provider_name=provider_name,
                backend_name=backend_name,
                circuits=circuit,
                shots=shots,
            )
    return dispatch, policy_data

We can now register this policy on the `QuantumExecutor`:

In [31]:
qe.add_policy(name="custom-multiplier", split_policy=split)

And then run the circuit. This time we run the experiment asynchronously.

In [32]:
res7 = qe.run_experiment(
    circuit=qc,
    shots=1024,
    backends=backends,
    split_policy="custom-multiplier",
    multiprocess=True,
    wait=True,
)

res7

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=59a55ce2-cdad-46e1-91d6-551e4244ac4b, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 508, '01': 13, '10': 15, '11': 488})], 'fake_oslo': [JobResult(job=Job(id=067d1d10-f9ae-4d5a-b114-e534d1612694, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 517, '01': 8, '10': 15, '11': 484})]}})

Finally, also in this case we can express a merge policy

In [33]:
res8 = qe.run_experiment(
    circuit=qc,
    shots=1024,
    backends=backends,
    split_policy="custom-multiplier",
    merge_policy="max_frequency",
    multiprocess=False,
    wait=True,
)

res8

MergedResultCollector(merged_results={'00': 1015}, initial_policy_data={}, final_policy_data={})